In [1]:
import snowflake.snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import pandas as pd
import numpy as np
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [2]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# # Current Environment Details
# print('User                        : {}'.format(snowflake_environment[0][0]))
# print('Role                        : {}'.format(snowflake_environment[0][1]))
# print('Database                    : {}'.format(snowflake_environment[0][2]))
# print('Schema                      : {}'.format(snowflake_environment[0][3]))
# print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
# print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
# print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

In [ ]:
%%writefile main.py
import streamlit as st
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import datetime as dt
# Snowpark for Python
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit,count
from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 

_, col, _ = st.columns([0.5,1,0.5])
question_list = ['query{}.tql'.format(x) for x in range(1,6)]
with col1:

    MetricSlider01 = st.selectbox("Pick your 1st metric", question_list)


In [5]:
session.sql('use schema snowflake_sample_data.tpcds_sf10Tcl;')
session.sql('''
with customer_total_return as
(select sr_customer_sk as ctr_customer_sk
,sr_store_sk as ctr_store_sk
,sum(SR_RETURN_AMT) as ctr_total_return
from store_returns
,date_dim
where sr_returned_date_sk = d_date_sk
and d_year =2000
group by sr_customer_sk
,sr_store_sk)
 select  c_customer_id
from customer_total_return ctr1
,store
,customer
where ctr1.ctr_total_return > (select avg(ctr_total_return)*1.2
from customer_total_return ctr2
where ctr1.ctr_store_sk = ctr2.ctr_store_sk)
and s_store_sk = ctr1.ctr_store_sk
and s_state = 'TN'
and ctr1.ctr_customer_sk = c_customer_sk
order by c_customer_id
 limit 100;
''').show()

2023-02-25 16:14:38.877 INFO    snowflake.connector.cursor: query: [with customer_total_return as (select sr_customer_sk as ctr_customer_sk ,sr_stor...]
2023-02-25 16:14:38.964 INFO    snowflake.connector.cursor: query execution done


SnowparkSQLException: (1304): 01aa931a-0d04-3c20-0002-fbda0001421a: 002003 (42S02): 01aa931a-0d04-3c20-0002-fbda0001421a: SQL compilation error:
Object 'STORE_RETURNS' does not exist or not authorized.